# Compare tau decay distributions between Tauola and IceCube simulations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyhepmc

## Load events from IceCube

In [3]:
particle_info = pd.read_csv('genie_tau_decays_particles.csv')

## Energy of tau neutrino

In [8]:
with pyhepmc.open("tauola_output.dat", "r") as f:
    evt = f.read()
    evt2 = f.read()

In [9]:
evt

<GenEvent momentum_unit=1, length_unit=0, event_number=0, particles=5, vertices=2, run_info=GenRunInfo(tools=[], weight_names=[], attributes={})>

In [10]:
evt2

<GenEvent momentum_unit=1, length_unit=0, event_number=1, particles=7, vertices=3, run_info=GenRunInfo(tools=[], weight_names=[], attributes={})>